<b>Community Detection using BigCLAM</b>

In [41]:
import networkx as nx 
import numpy as np
import pandas as pd
import math
import itertools as IT
import scipy
from random import randint, choice
import random
import matplotlib.pyplot as plt
import warnings

***Loading of the Youtube Edge List Data***

In [3]:
G = nx.read_edgelist("/content/YouTube.edgelist", nodetype=int)

In [22]:
#Number of nodes and edges in the given graph
V = G.number_of_nodes()
E = G.number_of_edges()
print("Number of Nodes: ",V)
print("Number of Edges: ",E)

Number of Nodes:  7675
Number of Edges:  35622


In [23]:
#Number of communities in the ground truth file
K = sum(1 for line in open('/content/groundtruth_communities.txt'))

**1. Factor matrix initialization:** 3 Matrix are initalized, 20_Percent_Seed, Neighborhood seed and a Random Initalization

In [24]:
#Matrix for 20 percent seed value
file_20percent = "/content/20percent_seed_communities.txt"
list_20percent = [] 
with open(file_20percent) as fp: 
  for line in fp: 
    list_20percent.append([int(i) for i in line.split()]) 

In [26]:
fmatrix_20percent = np.zeros((V,K)) 
for i in range(len(list_20percent)): 
  for j in list_20percent[i]: 
    fmatrix_20percent[j][i] = 1 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [28]:
#Conducatnace
for i in range(len(fmatrix_20percent)): 
  value_minimum = np.inf 
  for j in range(len(fmatrix_20percent[0])): 
    conduct = nx.conductance(G, (list_20percent[j]+list(G.neighbors(i))+[i])) 
    if value_minimum > conduct: 
      value_minimum = conduct 
      conduct_minimum = j 
    for k in list(G.neighbors(i))+[i]: 
      fmatrix_20percent[k][conduct_minimum] = 1
fmatrix_20percent 

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [44]:
neighSeed_file = "/content/neighborhood_seeds.txt" 
neighSeed_list = [] 
with open(neighSeed_file) as fn: 
  for line in fn: 
    neighSeed_list.append([int(i) for i in line.split()]) 

In [45]:
fmatrix_neighseed = np.zeros((V,K)) 
for i in range(len(neighSeed_list)): 
  for j in neighSeed_list[i]:  
    fmatrix_neighseed[j][i] = 1

In [46]:
for i in range(len(fmatrix_neighseed)): 
  value_minimum = np.inf 
  for j in range(len(fmatrix_neighseed[0])): 
    conduct = nx.conductance(G, (neighSeed_list[j]+list(G.neighbors(i))+[i])) 
    if value_minimum > conduct: 
      value_minimum = conduct 
      conduct_minimum = j 
    for k in list(G.neighbors(i))+[i]: 
      fmatrix_neighseed[k][conduct_minimum] = 1
fmatrix_neighseed

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [73]:
fmatrix_random = np.zeros((nodes_V,K_Comm))
#Random[0,1] Seeds Initalization
for i in range(V):
    for j in range(29):
        fmatrix_random[i][j] = random.random()
        fmatrix_random[i][j] = round(fmatrix_random[i][j],6)

**2. Matrix factorization: bigCLAM Algorithm**

In [42]:
warnings.filterwarnings("ignore")

In [48]:
def matrixFactorization(data_matrix):
    p_change=0
    Mat = data_matrix
    for iter in range(300):
        sumNodes = Mat.sum(axis=0) 
        for i in range(len(Mat)): 
            neighVals = list(G.neighbors(i)) 
            sum_V = [0]*K 
            d_val = [0]*K 
            for j in neighVals:
                if (1-np.exp(-np.matmul(Mat[i],Mat[j].transpose()))): 
                    #Delta function
                    d_update = np.exp(-np.matmul(Mat[i],Mat[j].transpose()))/(1-np.exp(-np.matmul(Mat[i],Mat[j].transpose())))
                else: 
                    d_update = 0 
                d_val = (Mat[j]*d_update) + d_val 
                sum_V = Mat[j] + sum_V 
            x = sumNodes - Mat[i] - sum_V 
            finalVal = d_val - x 
            change = 0.1*sum(finalVal)/sum(Mat[i])
            
            #Check for learning rate
            if change < 0.001:
                p_change+=1
            Mat[i] = Mat[i] + (0.001*finalVal) 
            Non_Neg_Vector = Mat[i] < 0 
            Mat[i][Non_Neg_Vector] = 0 
            if p_change == nodes_V:
                break        
    return Mat

In [49]:
BC_FMatrix_20Percent = matrixFactorization(fmatrix_20percent)

In [50]:
BC_FMatrix_20Percent

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
BC_FMatrix_NeighSeed = matrixFactorization(fmatrix_neighseed)

In [65]:
BC_FMatrix_NeighSeed

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [74]:
BC_FMatrix_Random = matrixFactorization(fmatrix_random)

In [75]:
BC_FMatrix_Random

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**3. Community Assignment**

In [51]:
delta = math.sqrt(-math.log(1-math.pow(10,-8)))
delta

0.00010000000050123797

In [55]:
def communityAssignment(data_matrix, data_SeedMatrix):
    matVal = data_matrix
    newMatix = data_SeedMatrix
    for i in range(len(matVal)):
        for j in range(len(matVal[0])):
            if matVal[i][j] >= delta:
                try:
                    newMatix[j][i] = 1
                except:
                    newMatix[j] = {}
                    newMatix[j][i] = 1
    return newMatix

In [56]:
def fileWrite(dataValues, file_name):
    fileName = file_name
    dataMatrix = dataValues
    fileN = open('/content/'+file_name,'w')
    for key,value in dataMatrix.items():
        fileN.write(str(key)+str("->"))
        if type(value) is dict:
            for k,v in value.items():
                lista = []
                lista.append(k)
            fileN.write(str(lista))
            fileN.write("\n")
    fileN.close()

In [57]:
newMat_20percent = dict()
resMat_20percent = communityAssignment(BC_FMatrix_20Percent,newMat_20percent)
name_20percent = 'Percent_20_Detected.txt'
fileWrite(resMat_20percent,name_20percent)

In [68]:
newMat_NeighSeed = dict()
resMat_NeighSeed = communityAssignment(BC_FMatrix_NeighSeed,newMat_NeighSeed)
name_NeighSeed = 'Neighor_Seed_Detected.txt'
fileWrite(resMat_NeighSeed,name_NeighSeed)

In [76]:
newMat_Random = dict()
resMat_Random = communityAssignment(BC_FMatrix_Random,newMat_Random)
name_Random = 'Random_Detected.txt'
fileWrite(resMat_Random,name_Random)

**4. Evaluation**


In [59]:
GC_Dict = dict()
i=0
GC_file = '/content/groundtruth_communities.txt'
with open(GC_file) as fp:
  for line in fp: 
    vals = [int(a) for a in line.split()]
    Result_Dict = {vals[i]: 1 for i in range(0, len(vals))}
    GC_Dict[i] = Result_Dict
    i+=1

In [62]:
def RecallFun(data_mat):
    data_Vals = data_mat
    finalVals = 0
    for community, nodes in GC_Dict.items():
        old = set(nodes.keys())
        if community in data_Vals:
            new = set(data_Vals[community].keys())
        else:
            new = set()
        tp = new & old
        finalVals += (len(tp)/len(old))
    finalVals = finalVals/K
    return finalVals